In [41]:
%%file ../libs/linearRegCostFn.m
function J = linearRegCostFn(X, y, theta, lambda)
    m = length(y);
    J = 0;

    error = X * theta - y;
    J = (error' * error) / (2*m);
    v = theta;
    v(1) = 0;

    reg = (lambda/(2*m)) * (v' * v);
    J = J + reg;
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/linearRegCostFn.m'.


In [45]:
%%file ../libs/linearRegGradFn.m
function grad = linearRegGradFn(X, y, theta, lambda)
    m = length(y);
    grad = zeros(size(theta));

    v = theta;
    v(1) = 0;

    grad = ((X * theta - y)' * X)' / m;
    reg = (lambda / m) * v;
    grad = grad + reg;
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/linearRegGradFn.m'.


In [14]:
%%file ../libs/linearRegCostNGradFunction.m
function [J, grad] = linearRegCostNGradFunction(X, y, theta, lambda)
    J = linearRegCostFn(X, y, theta, lambda);
    grad = linearRegGradFn(X, y, theta, lambda);
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/linearRegCostNGradFunction.m'.


In [2]:
%%file ../libs/trainLinearReg.m
function [theta] = trainLinearReg(X, y, lambda)
    initial_theta = zeros(size(X, 2), 1);
    costFn = @(t) linearRegCostNGradFunction(X, y, t, lambda);
    options = optimset('MaxIter', 200, 'GradObj', 'on');
    theta = fmincg(costFn, initial_theta, options);
    % theta = fminunc(costFn, initial_theta, options);
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/trainLinearReg.m'.


In [51]:
%%file ../libs/learningCurve.m
function [error_train, error_val] = learningCurve(X, y, Xval, yval, lambda)
    m = size(X, 1);
    error_train = zeros(m, 1);
    error_val = zeros(m, 1);

    for i = 1:m
        theta = trainLinearReg(X(1:i, :), y(1:i), lambda);
        error_train(i) = linearRegCostFn(X(1:i, :), y(1:i), theta, 0);
        error_val(i) = linearRegCostFn(Xval, yval, theta, 0);
    end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/learningCurve.m'.


In [76]:
%%file ../libs/RndlearningCurve.m
function [error_train, error_val] = RndlearningCurve(X, y, Xval, yval, lambda)
    m = size(X, 1);
    error_train = zeros(m, 1);
    error_val = zeros(m, 1);

    for i = 1:m
        error_sample_train = zeros(i, 1);
        error_sample_val = zeros(i, 1);
        for j=1:m/i
            train_seq = randperm(m, i);
            theta = trainLinearReg(X(train_seq, :), y(train_seq), lambda);
            error_sample_train(i) = linearRegCostFn(X(train_seq, :), y(train_seq), theta, 0);
            error_sample_val(i) = linearRegCostFn(Xval, yval, theta, 0);
        end
        error_train(i) = mean(error_sample_train);
        error_val(i) = mean(error_sample_val);
    end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/RndlearningCurve.m'.


In [55]:
%%file ../libs/polyFeatures.m
function [X_poly] = polyFeatures(X, p)
    m = size(X, 1);
    X_poly = zeros(m, p);

    for i = 1:p
        X_poly(:, i) = X(:, 1) .^ i;
    end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/polyFeatures.m'.


In [60]:
%%file ../libs/featureNormalize.m
function [X_norm, mu, sigma] = featureNormalize(X)
    mu = mean(X);
    X_norm = bsxfun(@minus, X, mu);

    sigma = std(X_norm);
    X_norm = bsxfun(@rdivide, X_norm, sigma);
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/featureNormalize.m'.


In [57]:
%%file ../libs/plotFit.m
function plotFit(min_x, max_x, mu, sigma, theta, p)
    hold on;

    x = (min_x - 15: 0.05 : max_x + 25)';
    X_poly = polyFeatures(x, p);
    X_poly = bsxfun(@minus, X_poly, mu);
    X_poly = bsxfun(@rdivide, X_poly, sigma);
    X_poly = [ones(size(x, 1), 1) X_poly];

    plot(x, X_poly * theta, '--', 'LineWidth', 2);

    hold off;
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/plotFit.m'.


In [59]:
%%file ../libs/validationCurve.m
function [lambda_vec, error_train, error_val] = validationCurve(X, y, Xval, yval)
    lambda_vec = [0 0.001 0.003 0.01 0.03 0.1 0.3 1 3 10]';

    error_train = zeros(length(lambda_vec), 1);
    error_val = zeros(length(lambda_vec), 1);

    for i = 1:length(lambda_vec)
        lambda = lambda_vec(i);
        theta = trainLinearReg(X, y, lambda);
        error_train(i) = linearRegCostFn(X, y, theta, 0);
        error_val(i) = linearRegCostFn(Xval, yval, theta, 0);
    end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/validationCurve.m'.


# Helper

In [28]:
%%file ../libs/fmincg.m
function [X, fX, i] = fmincg(f, X, options, P1, P2, P3, P4, P5)
    % Minimize a continuous differentialble multivariate function. Starting point
    % is given by "X" (D by 1), and the function named in the string "f", must
    % return a function value and a vector of partial derivatives. The Polack-
    % Ribiere flavour of conjugate gradients is used to compute search directions,
    % and a line search using quadratic and cubic polynomial approximations and the
    % Wolfe-Powell stopping criteria is used together with the slope ratio method
    % for guessing initial step sizes. Additionally a bunch of checks are made to
    % make sure that exploration is taking place and that extrapolation will not
    % be unboundedly large. The "length" gives the length of the run: if it is
    % positive, it gives the maximum number of line searches, if negative its
    % absolute gives the maximum allowed number of function evaluations. You can
    % (optionally) give "length" a second component, which will indicate the
    % reduction in function value to be expected in the first line-search (defaults
    % to 1.0). The function returns when either its length is up, or if no further
    % progress can be made (ie, we are at a minimum, or so close that due to
    % numerical problems, we cannot get any closer). If the function terminates
    % within a few iterations, it could be an indication that the function value
    % and derivatives are not consistent (ie, there may be a bug in the
    % implementation of your "f" function). The function returns the found
    % solution "X", a vector of function values "fX" indicating the progress made
    % and "i" the number of iterations (line searches or function evaluations,
    % depending on the sign of "length") used.
    %
    % Usage: [X, fX, i] = fmincg(f, X, options, P1, P2, P3, P4, P5)
    %
    % See also: checkgrad 
    %
    % Copyright (C) 2001 and 2002 by Carl Edward Rasmussen. Date 2002-02-13
    %
    %
    % (C) Copyright 1999, 2000 & 2001, Carl Edward Rasmussen
    % 
    % Permission is granted for anyone to copy, use, or modify these
    % programs and accompanying documents for purposes of research or
    % education, provided this copyright notice is retained, and note is
    % made of any changes that have been made.
    % 
    % These programs and documents are distributed without any warranty,
    % express or implied.  As the programs were written for research
    % purposes only, they have not been tested to the degree that would be
    % advisable in any important application.  All use of these programs is
    % entirely at the user's own risk.
    %
    % [ml-class] Changes Made:
    % 1) Function name and argument specifications
    % 2) Output display
    %

    % Read options
    if exist('options', 'var') && ~isempty(options) && isfield(options, 'MaxIter')
        length = options.MaxIter;
    else
        length = 100;
    end


    RHO = 0.01;                            % a bunch of constants for line searches
    SIG = 0.5;       % RHO and SIG are the constants in the Wolfe-Powell conditions
    INT = 0.1;    % don't reevaluate within 0.1 of the limit of the current bracket
    EXT = 3.0;                    % extrapolate maximum 3 times the current bracket
    MAX = 20;                         % max 20 function evaluations per line search
    RATIO = 100;                                      % maximum allowed slope ratio

    argstr = ['feval(f, X'];                      % compose string used to call function
    for i = 1:(nargin - 3)
        argstr = [argstr, ',P', int2str(i)];
    end
    argstr = [argstr, ')'];

    if max(size(length)) == 2, red=length(2); length=length(1); else red=1; end
    S=['Iteration '];

    i = 0;                                            % zero the run length counter
    ls_failed = 0;                             % no previous line search has failed
    fX = [];
    [f1 df1] = eval(argstr);                      % get function value and gradient
    i = i + (length<0);                                            % count epochs?!
    s = -df1;                                        % search direction is steepest
    d1 = -s'*s;                                                 % this is the slope
    z1 = red/(1-d1);                                  % initial step is red/(|s|+1)

    while i < abs(length)                                      % while not finished
        i = i + (length>0);                                      % count iterations?!

        X0 = X; f0 = f1; df0 = df1;                   % make a copy of current values
        X = X + z1*s;                                             % begin line search
        [f2 df2] = eval(argstr);
        i = i + (length<0);                                          % count epochs?!
        d2 = df2'*s;
        f3 = f1; d3 = d1; z3 = -z1;             % initialize point 3 equal to point 1
        if length>0, M = MAX; else M = min(MAX, -length-i); end
        success = 0; limit = -1;                     % initialize quanteties
        while 1
            while ((f2 > f1+z1*RHO*d1) || (d2 > -SIG*d1)) && (M > 0) 
                limit = z1;                                         % tighten the bracket
                if f2 > f1
                  z2 = z3 - (0.5*d3*z3*z3)/(d3*z3+f2-f3);                 % quadratic fit
                else
                  A = 6*(f2-f3)/z3+3*(d2+d3);                                 % cubic fit
                  B = 3*(f3-f2)-z3*(d3+2*d2);
                  z2 = (sqrt(B*B-A*d2*z3*z3)-B)/A;       % numerical error possible - ok!
                end
                if isnan(z2) || isinf(z2)
                  z2 = z3/2;                  % if we had a numerical problem then bisect
                end
                z2 = max(min(z2, INT*z3),(1-INT)*z3);  % don't accept too close to limits
                z1 = z1 + z2;                                           % update the step
                X = X + z2*s;
                [f2 df2] = eval(argstr);
                M = M - 1; i = i + (length<0);                           % count epochs?!
                d2 = df2'*s;
                z3 = z3-z2;                    % z3 is now relative to the location of z2
            end
            if f2 > f1+z1*RHO*d1 || d2 > -SIG*d1
                break;                                                % this is a failure
            elseif d2 > SIG*d1
                success = 1; break;                                             % success
            elseif M == 0
                break;                                                          % failure
            end
            A = 6*(f2-f3)/z3+3*(d2+d3);                      % make cubic extrapolation
            B = 3*(f3-f2)-z3*(d3+2*d2);
            z2 = -d2*z3*z3/(B+sqrt(B*B-A*d2*z3*z3));        % num. error possible - ok!
            if ~isreal(z2) || isnan(z2) || isinf(z2) || z2 < 0   % num prob or wrong sign?
                if limit < -0.5                               % if we have no upper limit
                    z2 = z1 * (EXT-1);                 % the extrapolate the maximum amount
                else
                    z2 = (limit-z1)/2;                                   % otherwise bisect
                end
            elseif (limit > -0.5) && (z2+z1 > limit)          % extraplation beyond max?
                z2 = (limit-z1)/2;                                               % bisect
            elseif (limit < -0.5) && (z2+z1 > z1*EXT)       % extrapolation beyond limit
                z2 = z1*(EXT-1.0);                           % set to extrapolation limit
            elseif z2 < -z3*INT
                z2 = -z3*INT;
            elseif (limit > -0.5) && (z2 < (limit-z1)*(1.0-INT))   % too close to limit?
                z2 = (limit-z1)*(1.0-INT);
            end
            f3 = f2; d3 = d2; z3 = -z2;                  % set point 3 equal to point 2
            z1 = z1 + z2; X = X + z2*s;                      % update current estimates
            [f2 df2] = eval(argstr);
            M = M - 1; i = i + (length<0);                             % count epochs?!
            d2 = df2'*s;
        end                                                      % end of line search

        if success                                         % if line search succeeded
            f1 = f2; fX = [fX' f1]';
            fprintf('%s %4i | Cost: %4.6e\r', S, i, f1);
            s = (df2'*df2-df1'*df2)/(df1'*df1)*s - df2;      % Polack-Ribiere direction
            tmp = df1; df1 = df2; df2 = tmp;                         % swap derivatives
            d2 = df1'*s;
            if d2 > 0                                      % new slope must be negative
                s = -df1;                              % otherwise use steepest direction
                d2 = -s'*s;    
            end
            z1 = z1 * min(RATIO, d1/(d2-realmin));          % slope ratio but max RATIO
            d1 = d2;
            ls_failed = 0;                              % this line search did not fail
        else
          X = X0; f1 = f0; df1 = df0;  % restore point from before failed line search
          if ls_failed || i > abs(length)          % line search failed twice in a row
              break;                             % or we ran out of time, so we give up
          end
          tmp = df1; df1 = df2; df2 = tmp;                         % swap derivatives
          s = -df1;                                                    % try steepest
          d1 = -s'*s;
          z1 = 1/(1-d1);                     
          ls_failed = 1;                                    % this line search failed
        end
        if exist('OCTAVE_VERSION')
            fflush(stdout);
        end
      end
end

Created file '/Users/jchien/workspace/courses/coursera_ml/ex5/octave/libs/fmincg.m'.
